Started once text preprocessing and vector database setup was finished :)

In [12]:
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain.llms import openai

class VectorDatabaseHandler:
    def __init__(self, model_name='all-MiniLM-L6-v2', persist_directory='vector_db'):
        # Initialize the SentenceTransformer model
        self.model = SentenceTransformer(model_name)

        # Set up Chroma vector store
        self.embedding_function = SentenceTransformerEmbeddingFunction(self.model)
        self.vector_store = Chroma(
            collection_name="document_collection",
            embedding_function=self.embedding_function,
            persist_directory=persist_directory  # Directory for persistent Chroma storage
        )

    def load_vector_database(self, filename='vector_database.pkl'):

        """Load the saved vector database from a pickle file."""
        with open(filename, 'rb') as file:
            vector_database = pickle.load(file)
        print(f"Vector database loaded from {filename}")

        return vector_database['embeddings'], vector_database['metadata']

    def save_vector_database(self, embeddings_list, metadata_list, filename='vector_database.pkl'):
        """Save embeddings and metadata to a pickle file."""
        vector_database = {
            'embeddings': embeddings_list,
            'metadata': metadata_list
        }
        with open(filename, 'wb') as file:
            pickle.dump(vector_database, file)
        print(f"Vector database saved to {filename}")

    def generate_embeddings(self, texts):
        """Generate embeddings for a list of texts."""
        return self.model.encode(texts, convert_to_numpy=True)

    def populate_vector_store_from_saved_data(self, filename='vector_database.pkl'):
        """Populate the vector store with data loaded from the saved vector database."""

        try: 
            # Load data from the saved pickle file
            embeddings, metadata = self.load_vector_database(filename)
            embeddings = [
                np.nan_to_num(embedding, nan=0.0, posinf=0.0, neginf=0.0)
                for embedding in embeddings
            ]

            # Ensure embeddings are in the correct format (list of lists)
            if isinstance(embeddings, np.ndarray):
                embeddings = embeddings.tolist()  # Convert to list if it's a numpy array
            elif not isinstance(embeddings, list):
                raise ValueError("Embeddings should be a list or numpy array.")

            # Ensure embeddings are not empty
            if isinstance(embeddings, np.ndarray):  # If embeddings is a NumPy array
                if len(embeddings)== 0:
                    raise ValueError("Embeddings are empty.")
            elif isinstance(embeddings, list):  # If embeddings is a list
                if len(embeddings) == 0:
                    raise ValueError("Embeddings are empty.")
            else:
                raise ValueError("Unknown type for embeddings. Expected list or numpy.ndarray.")

            # old version of flattening
            # embeddings = [embedding if isinstance(embedding, list) else embedding.tolist() for embedding in embeddings]
            # new version of flattering : ensure each embedding is a list (flattening if necessary)
            embeddings = [embedding.flatten().tolist() if isinstance(embedding, np.ndarray) else embedding for embedding in embeddings]

            # Ensure embeddings is not empty
            if len(embeddings)==0:
                raise ValueError("Embeddings are empty.")

            # Ensure metadata is correctly structured
            if not metadata:
                raise ValueError("Metadata is empty.")

            # Ensures original text exists
            texts = [meta['original_text'] for meta in metadata if 'original_text' in meta]
            if not texts:
                raise ValueError("No 'original_text' key found in metadata.")

            # Ensures embeddings are numeric
            for idx, embedding in enumerate(embeddings):
                if not all(isinstance(val, (int, float, np.floating, np.integer)) for val in embedding):
                    raise ValueError(f"Embedding at index {idx} contains non-numeric values: {embedding}")

            # Add embeddings and metadata to the vector store
            self.vector_store.add_texts(
                texts=[meta['original_text'] for meta in metadata],  # Assuming original_text is part of the metadata
                metadatas=metadata,
                embeddings=embeddings  # Now we directly pass embeddings
            )

            # Add texts and embeddings one by one to debug
            # for i in range(len(metadata)):
            #     try:
            #         self.vector_store.add_texts(
            #             texts=[metadata[i]['original_text']],  # Add a single text at a time
            #             metadatas=[metadata[i]],  # Add the corresponding metadata
            #             embeddings=[embeddings[i]]  # Add a single embedding
            #         )
            #     except Exception as e:
            #         print(f"Error adding item {i}: {e}")

            self.vector_store.persist()
            print("Vector store populated with data from saved file.")

            # debugging the data structure
            print(f"Embeddings type: {type(embeddings)}, length: {len(embeddings)}")
            print(f"Metadata type: {type(metadata[:3])}, length: {len(metadata)}")
        except ValueError as e:
            print(f"ValueError occured: {e}")
        except Exception as e:
            print(f"An error occured: {e}")

        # test the data after loading
        print(f"Loaded embeddings: {embeddings[:3]}")  # Preview first 3 embeddings
        print(f"Loaded metadata: {metadata[:3]}")    # Preview first 3 metadata entries
        print(f"Embeddings preview: {embeddings[:3]}")  # Check first 3 embeddings
        # does not print so len of embeddings and metadata match so not the problem
        if len(embeddings) != len(metadata):
            raise ValueError(f"Number of embeddings ({len(embeddings)}) does not match number of metadata ({len(metadata)}).")
        # both print class list so that's not the problem
        print(f"Type of embeddings: {type(embeddings)}")
        print(f"Type of metadata: {type(metadata)}")

    def handle_query(self, user_query, k=2):
        """Handles a user query by retrieving relevant documents and generating a response."""
        # Perform similarity search in the vector database
        results = self.vector_store.similarity_search(
            query=user_query,
            k=k  # Number of relevant documents to retrieve
        )

        # Extract relevant documents and metadata
        retrieved_docs = [result['text'] for result in results]
        retrieved_metadata = [result['metadata'] for result in results]

        # Construct a simple prompt without using PromptTemplate
        prompt = f"""
        You are an AI assistant. Below are some relevant documents retrieved based on a user's query.
        Use this information to generate a concise and helpful response.

        Relevant Documents:
        {retrieved_docs}

        User Query:
        {user_query}

        Your Response:
        """

        # Initialize an LLM (e.g., OpenAI's GPT) for response generation
        llm = openai(model_name="gpt-4")  # Replace with your preferred LLM model or API key setup

        # Generate the response
        response = llm(prompt)
        return response

class SentenceTransformerEmbeddingFunction:
    def __init__(self, model):
        self.model = model

    # Embedding method for documents
    def embed_documents(self, texts):
        embeddings = self.model.encode(texts, convert_to_numpy=True)
        return embeddings

    # Embedding method for queries
    def embed_query(self, query):
        embedding = self.model.encode([query], convert_to_numpy=True)
        return embedding


# Example usage
if __name__ == "__main__":
    # Initialize VectorDatabaseHandler
    vector_db_handler = VectorDatabaseHandler()

    # Step 1: Populate the vector store with the data from the saved vector database
    vector_db_handler.populate_vector_store_from_saved_data()

    # Step 2: Define a function to process a user query and generate a response
    user_query = "how to make chicken"
    response = vector_db_handler.handle_query(user_query)

    # Step 3: Display the response
    print(f"User Query: {user_query}")
    print(f"AI Response:\n{response}")

Vector database loaded from vector_database.pkl
ValueError occured: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Loaded embeddings: [[-0.028531987220048904, -0.017205100506544113, -0.01807147078216076, -0.025295797735452652, -0.02556699886918068, -0.03505215793848038, -0.005073211621493101, 0.037991516292095184, 0.05599883571267128, 0.05340452119708061, 0.02292126603424549, 0.01862766221165657, -0.08795951306819916, -0.012520243413746357, 0.012860649265348911, -0.021059958264231682, 0.018680723384022713, 0.05720271170139313, -0.09307244420051575, -0.004500418435782194, 0.1516454517841339, -0.01776779256761074, -0.12361284345388412, 0.027725908905267715, 0.04199016094207764, -0.030380191281437874, -0.03875354677438736, 0.0005685559008270502, -0.05026441812515259, 0.08586353063583374, 0.02625224180519581, 0.014997834339737892, -0.022484201937913895, 0.04712257534265518, 0.024279026314616203, 0.047875892370939255, 0.013904040679335594, 0.0032

ValueError: Expected each value in the embedding to be a int or float, got an embedding with ['ndarray'] - [[-6.07468039e-02 -4.96362261e-02 -3.19795497e-02 -2.53695971e-03
  -3.09494585e-02 -3.39189311e-03  2.99431756e-02 -3.21766213e-02
   1.97168719e-02 -1.28607675e-02 -1.46663059e-02 -4.06498201e-02
  -3.48833241e-02 -3.47304493e-02 -3.92895080e-02 -6.10014647e-02
   2.16313563e-02 -7.56658334e-03 -2.43046340e-02 -8.01153854e-02
  -2.52891816e-02 -8.94731283e-02  6.37236312e-02 -2.95828450e-02
   1.13018928e-02  4.55256412e-03  5.39371595e-02  3.66967432e-02
   2.76301056e-02 -1.12386756e-02  8.90882611e-02 -3.67554352e-02
  -5.01450012e-03 -1.47742992e-02 -4.10877839e-02 -5.58128767e-03
   6.59301784e-03 -2.65122368e-03  7.13664368e-02  3.71642932e-02
   4.88988310e-02 -6.09838292e-02  9.86229926e-02 -3.71611193e-02
   9.10603032e-02  4.72435318e-02  2.93108169e-02  4.37863022e-02
   4.92542349e-02 -1.15918107e-01 -5.89001328e-02 -2.56316252e-02
  -5.35212606e-02 -7.29585811e-02  